# Create a Local Docker Image
In this section, we will create an IoT Edge module, a Docker container image with an HTTP web server that has a scoring REST endpoint.

## Get Global Variables

In [13]:
import sys
sys.path.append('../../../common')
from env_variables import *

## Create Web Application & Inference Server for Our ML Solution

In [14]:
%%writefile $isSolutionPath/app.py
import threading
import cv2
import numpy as np
import io
import onnxruntime
import json
import logging
import linecache
import sys
from score import MLModel, PrintGetExceptionDetails
from flask import Flask, request, jsonify, Response

logging.basicConfig(level=logging.DEBUG)

app = Flask(__name__)
inferenceEngine = MLModel()

@app.route("/score", methods = ['POST'])
def scoreRRS():
    global inferenceEngine

    try:
        # get request as byte stream
        reqBody = request.get_data(False)

        # convert from byte stream
        inMemFile = io.BytesIO(reqBody)

        # load a sample image
        inMemFile.seek(0)
        fileBytes = np.asarray(bytearray(inMemFile.read()), dtype=np.uint8)
        cvImage = cv2.imdecode(fileBytes, cv2.IMREAD_COLOR)

        # Infer Image
        detectedObjects = inferenceEngine.Score(cvImage)

        if len(detectedObjects) > 0:
            respBody = {                    
                        "inferences" : detectedObjects
                    }

            respBody = json.dumps(respBody)
            
            logging.info("[AI EXT] Sending response.")
            return Response(respBody, status= 200, mimetype ='application/json')
        else:
            logging.info("[AI EXT] Sending empty response.")
            return Response(status= 204)

    except:
        PrintGetExceptionDetails()
        return Response(response='Exception occured while processing the image.', status=500)
    
@app.route("/")
def healthy():
    return "Healthy"

# About
@app.route('/about', methods = ['GET'])
def about_request():
    global inferenceEngine
    return inferenceEngine.About()

if __name__ == "__main__":
    app.run(host='127.0.0.1', port=5444)

Overwriting inferenceserver/app.py


In the cell above, 5444 is the internal port of the webserver app that listens the requests. Next, we will map it to different ports to expose it externally.

In [15]:
%%writefile $isSolutionPath/wsgi.py
from app import app as application

def create():
    application.run(host='127.0.0.1', port=5444)

Overwriting inferenceserver/wsgi.py


In [16]:
import os
os.makedirs(os.path.join(isSolutionPath, "nginx"), exist_ok=True)

The exposed port of the web app is now 5001, while the internal one is still 5444.

In [17]:
%%writefile $isSolutionPath/nginx/app
server {
    listen 5001;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5444;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

Overwriting inferenceserver/nginx/app


In [18]:
%%writefile $isSolutionPath/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

Overwriting inferenceserver/gunicorn_logging.conf


In [19]:
%%writefile $isSolutionPath/kill_supervisor.py
import sys
import os
import signal

def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('[AI EXT] READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('[AI EXT] Terminating supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('[AI EXT] Could not terminate supervisor: ' + e.strerror + '\n')
            write_stdout('[AI EXT] RESULT 2\nOK')

main()

Overwriting inferenceserver/kill_supervisor.py


In [20]:
import os
os.makedirs(os.path.join(isSolutionPath, "etc"), exist_ok=True)

In [21]:
%%writefile $isSolutionPath/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true                ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/isserver
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/isserver
events=PROCESS_STATE_FATAL
priority=2

Overwriting inferenceserver/etc/supervisord.conf


## Create a Docker File to Containerize the ML Solution and Web App Server



In [22]:
%%writefile $isSolutionPath/Dockerfile
FROM mcr.microsoft.com/azureml/onnxruntime:latest-cuda

ARG WORK_DIR=/isserver
WORKDIR ${WORK_DIR}

# Copy the app file
COPY . ${WORK_DIR}/
COPY etc /etc

# Install runit, python, nginx, and necessary python packages
RUN apt-get update && apt-get install -y --no-install-recommends \
    python3-pip python3-dev libglib2.0-0 libsm6 libxext6 libxrender-dev nginx supervisor python3-setuptools \
    && cd /usr/local/bin \
    && ln -s /usr/bin/python3 python \
    && pip3 install --upgrade pip \
    && pip install numpy onnxruntime-gpu flask pillow gunicorn opencv-python json-logging-py \
    && apt-get clean \
    && apt-get update && apt-get install -y --no-install-recommends \
    wget runit nginx \
    && rm -rf /var/lib/apt/lists/* \
    && apt-get clean \
    && rm /etc/nginx/sites-enabled/default \
    && cp /isserver/nginx/app /etc/nginx/sites-available/ \
    && ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/

EXPOSE 5001
CMD ["supervisord", "-c", "/isserver/etc/supervisord.conf"]

Overwriting inferenceserver/Dockerfile


## Create a Local Docker Image
Finally, we will create a Docker image locally. We will later host the image in a container registry like Docker Hub, Azure Container Registry, or a local registry.

To run the following code snippet, you must have the pre-requisities mentioned in [the requirements page](../../../common/requirements.md). Most notably, we are running the `docker` command without `sudo`.

> <span>[!WARNING]</span>
> Please ensure that Docker is running before executing the cell below. Execution of the cell below may take several minutes. 

In [23]:
!sudo docker build -t $containerImageName --file ./$isSolutionPath/Dockerfile ./$isSolutionPath

Sending build context to Docker daemon  247.9MB
Step 1/8 : FROM mcr.microsoft.com/azureml/onnxruntime:latest-cuda
 ---> 9d181493b8a0
Step 2/8 : ARG WORK_DIR=/isserver
 ---> Using cache
 ---> 576f492c091d
Step 3/8 : WORKDIR ${WORK_DIR}
 ---> Using cache
 ---> 877ff93bb24f
Step 4/8 : COPY . ${WORK_DIR}/
 ---> Using cache
 ---> 01944304832a
Step 5/8 : COPY etc /etc
 ---> Using cache
 ---> 01b2d2257d77
Step 6/8 : RUN apt-get update && apt-get install -y --no-install-recommends     python3-pip python3-dev libglib2.0-0 libsm6 libxext6 libxrender-dev nginx supervisor python3-setuptools     && cd /usr/local/bin     && ln -s /usr/bin/python3 python     && pip3 install --upgrade pip     && pip install numpy onnxruntime-gpu flask pillow gunicorn opencv-python json-logging-py     && apt-get clean     && apt-get update && apt-get install -y --no-install-recommends     wget runit nginx     && rm -rf /var/lib/apt/lists/*     && apt-get clean     && rm /etc/nginx/sites-enabled/default     && cp /isser

## Next Steps
If all the code cells above have successfully finished running, return to the Readme page to continue.   